<a href="https://colab.research.google.com/github/FawneeCakes/credit-risk-classification/blob/main/credit_risk_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,392 kB]
Fetched 1,652 kB in 4s (470 kB/s)
Reading package lists... Done


In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
# Import the train_test_learn module
from sklearn.model_selection import train_test_split
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# **Split the Data into Training and Testing Sets**

## **Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.**

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data = Path('Resources/lending_data.csv')
lending_df = pd.read_csv(data)

# Review the DataFrame (displayed both head and tail to see if 'loan_status' have both 0's and 1's)
display(lending_df.head())
display(lending_df.tail())

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1
77535,15600.0,9.742,72300,0.585062,9,2,42300,1


# **Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.**

In [5]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns=['loan_status'])

In [6]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [7]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


In [8]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

# **Step 3: Split the data into training and testing datasets by using `train_test_split`.**

In [9]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,
                                                    random_state=1,
                                                    )

# **Create a Logistic Regression Model with the Original Data**

**Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).**

In [10]:
# Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(random_state=1)
classifier = LogisticRegression(solver='lbfgs',
                                max_iter = 200,
                                random_state=1)

# Fit the model using training data
lr_model = classifier.fit(X_train, y_train)

**Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.**

In [11]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


In [12]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9442676901753825

**Step 3: Evaluate the model’s performance by doing the following:**

* Generate a confusion matrix.  

* Print the classification report.  

In [13]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,18679,80
Actual 1,67,558


In [14]:
# Print the classification report for the model
print("Classification Report")
target_names = ["healthy loan","high-risk loan"]
print(classification_report(y_test, predictions, target_names = target_names))

Classification Report
                precision    recall  f1-score   support

  healthy loan       1.00      1.00      1.00     18759
high-risk loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



# **Step 4: Answer the following question.**

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** WRITE YOUR ANSWER HERE!



*  Precision quantifies the accuracy of predictions for a particular label. Specifically, for the “healthy loan” class, the precision is 1.00, signifying perfect precision. However, for the “high-risk loan” class, the precision stands at 0.87, indicating that 87% of the predicted high-risk loans are accurate

*   Recall, also referred to as sensitivity or the true positive rate, quantifies the proportion of correctly predicted instances out of the total actual instances for a specific label. Specifically: For the “healthy loan” class, the recall is 1.00, indicating that the model captures all actual healthy loans. Meanwhile, for the “high-risk loan” class, the recall stands at 0.89, meaning that the model correctly identifies 89% of the high-risk loans


*   The F1-score strikes a balance between precision and recall. For the “healthy loan” class, the F1-score is 1.00, indicating perfect performance.
In contrast, the F1-score for the “high-risk loan” class is 0.88, representing a good balance between precision and recall.

*   The model’s accuracy score is 99%, indicating exceptional performance overall.

One potential improvement is to reduce the number of false negatives (currently at 67). In situations like identifying high-risk loans or detecting fraud, minimizing false negatives is crucial. Doing so increases the chances of correctly identifying positive instances and allows us to take appropriate actions based on accurate predictions

Resources: In class work and solutions; Stack overflow: Logistic regression python solvers' definitions, Fine-tuning parameters in Logistic Regression; Youtube: Scikit-learn Crash Course - Machine Learning Library for Python, Scikit-Learn Course - Machine Learning in Python Tutorial

